<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Generate_text_from_local_docs_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generative AI - Aumented with documents

In [ ]:
!pip install gpt4all sentence-transformers

In [9]:
from sentence_transformers import util, SentenceTransformer
from gpt4all import GPT4All
import sqlite3
import numpy as np

In [7]:
sent_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#model = GPT4All(model_path = "e:\Downloads", model_name="Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf")
## Not sure where collab stores models. You can upload a model to your drive and then
## load it e.g. above
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")

Downloading: 100%|██████████| 1.98G/1.98G [01:14<00:00, 26.5MiB/s]
Verifying: 100%|██████████| 1.98G/1.98G [00:06<00:00, 287MiB/s]


In [13]:
question = "How should I store Bananas?"

In [14]:
## 100 tokens is ~ 3 sentences
DB = "/content/drive/MyDrive/12-4-24-all-mini-token-100.db"

# just top score for now
def get_relevant_doc(question=["No question"], model=None, n=1, db=DB):

    q = model.encode(question)
    score = 0
    doc = None
    conn = sqlite3.connect(db)

    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    cursor.execute('SELECT * FROM documents')
    for row in cursor:
        if util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy() > score:
            doc = row
            score += util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

    if score < 0.5:
        return -99

    return doc[0]

# get the most relevant document
doc_id = get_relevant_doc(question = question, model=sent_model)

In [15]:
doc_id

3482

In [27]:
# Establish a connection to the SQLite database
conn = sqlite3.connect(DB)

# Create a cursor object
cursor = conn.cursor()

retrieved = cursor.execute(f"""SELECT * FROM documents WHERE id = {doc_id}""").fetchone()[3]

In [29]:
## The porompt templates can be found from the models website or using gpt4all gui
## Also are usually supplied int he docs of particular model

x = f"""### System:
You are an AI assistant that helps to answer questions as best you can and incorporate user input.

### User:
How Should I store Bananas?

### Input:
Additional context {retrieved}

### Response:
"""

In [28]:
retrieved

'. Visual quality was maintained longer where the fruit was kept in polyethylene bags. It is not appropriate to store bananas in the fridge as they are chilling-sensitive, and their skins become blackened, but the coolest place in the home is the best storage location and the fruit should be kept in bags. Bananas sold in the UK are mostly sourced from Central and South America. They are harvested while still in a pre-climacteric (green) condition and shipped to the UK where'

In [41]:
x = f"""### System:
You are an sophisticated AI assistant that helps to answer questions as best you can.

### User:
How Should I store Bananas?

### System:
Based on the retrieved information from the PDF, here are the relevant excerpts:

{retrieved}

Please answer the user's question, integrating insights from these excerpts and your general knowledge. Limit your answer to Three sentences.

### Response:
"""

In [38]:
response = model.generate(prompt="How Should I store Bananas?", temp=0)
print(response)


Bananas are a tropical fruit that require warm, humid conditions to grow and ripen properly. They should be stored in a cool, dry place away from direct sunlight. Here's how you can store bananas:
1. Store them at room temperature or in the refrigerator. Avoid storing them near fruits with strong odors as they may spoil faster.
2. Use a banana storage bag or wrap to keep them fresh for up to two weeks. You can also use a breathable produce wrap or a reusable container filled with water to help keep them moist.
3. If you're not going to eat the bananas within a few days, consider freezing them. Remove any damaged or moldy fruit before freezing and defrost them in the refrigerator.


In [42]:
response = model.generate(prompt=x, temp=0)
print(response)

 How should I store bananas? It is recommended to store them in polyethylene bags in a cool and dark place, such as the refrigerator or pantry. Bananas sold in the UK are mostly sourced from Central and South America and are harvested while still in a pre-climacteric (green) condition. They should not be stored at room temperature or in direct sunlight as it can cause them to blacken and spoil quickly. Additionally, bananas should not be washed until they are fully ripe as the natural sugars in their skin can become damaged during washing.
